# Libraries

In [ ]:
# numerical analysis
import numpy as np
# processing and storing in dataframe
import pandas as pd

# basic plotting
import matplotlib.pyplot as plt
# advanced plotting
import seaborn as sns
# interactive plotting
import plotly.express as px

# dealing with geographic data
import geopandas as gpd
# to get geolocation 
from geopandas.tools import geocode

# register the converters:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# plotly offline
import plotly
plotly.offline.init_notebook_mode(connected=True)

# Utility Functions

In [ ]:
def plot_cal(title, start, end):
    '''Plot incubation period'''
    inc_day = []
    for i in range(1, 29):
        if(i>=start and i<=end):
            inc_day.append(1)
        else:
            inc_day.append(0)
            
    inc_day = np.array(inc_day)
    inc_day = inc_day.reshape(4, 7)
    
    fig, ax = plt.subplots(figsize=(6, 3))
    ax=sns.heatmap(inc_day, linewidths=2, square=True, 
                   xticklabels='', yticklabels='', ax=ax,
                   cbar=False, cmap=['whitesmoke','royalblue'])
    ax.set_title(title, fontdict={'fontsize':16}, loc='left', pad=15)
    
    plt.show()

In [ ]:
def plot_barh(col):
    
    temp_df = latest.sort_values(col, ascending=False).head(10)
    temp_df = temp_df[temp_df[col]!=0]
    
    hover_data = ['Cumulative total cases', 'No. of deaths', 'Case fatalities ratio (%)']
    
    fig =  px.bar(temp_df, x=col, y='Country/Region', orientation='h', color='Country/Region', 
                  text=col, title=col, width=700, hover_data = hover_data,
                  color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_traces(textposition='auto')
    fig.update_layout(xaxis_title="", yaxis_title="", showlegend=False,
                      uniformtext_minsize=8, uniformtext_mode='hide')
    fig.show()

In [ ]:
def plot_pie(col1, col2, title, pal):

    temp = latest[[col1, col2]].sum()
    temp = pd.DataFrame(temp).reset_index()
    temp.columns = ['Column', 'Value']
    
    fig = px.sunburst(temp, path=['Column'], values='Value',
                      color_discrete_sequence=pal, title=title)
    fig.data[0].textinfo = 'label+text+value+percent root'
    fig.show()

# Dataset

### List of files

In [ ]:
# list of files
!ls -lt ../input/sars-outbreak-2003-complete-dataset

### Import files

In [ ]:
# importing daywise dataset
df = pd.read_csv('../input/sars-outbreak-2003-complete-dataset/sars_2003_complete_dataset_clean.csv', parse_dates=['Date'])
# rename columns
df.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']
# active no. of cases
df['Active'] = df['Cases'] - df['Deaths'] - df['Recovered']
# first few rows
df.head()

In [ ]:
# day wise data
day_wise = df.groupby(['Date'])['Cases', 'Deaths', 'Recovered', 'Active'].sum()
# reset index
day_wise = day_wise.reset_index()
# first few rows
day_wise.head()

In [ ]:
# importing summary dataset
latest = pd.read_csv('../input/sars-outbreak-2003-complete-dataset/summary_data_clean.csv')
# fix datetime columns
cols = ['Date onset first probable case', 'Date onset last probable case']
for col in cols:
    latest[col] = pd.to_datetime(latest[col])
cols = ['Number of Imported cases', 'Percentage of Imported cases']
for col in cols:
    latest[col] = latest[col].fillna(0)
# new columns
latest['Number of locally transmitted cases'] = latest['Cumulative total cases'] - latest['Number of Imported cases']
latest['Percentage of locally transmitted cases'] = 100 - latest['Percentage of Imported cases']
latest['Number of non HCW affected'] = latest['Cumulative total cases'] - latest['Number of HCW affected']
latest['Percentage of non HCW affected'] = 100 - latest['Percentage of HCW affected']
latest['Non fatality case (%)'] = 100 - latest['Case fatalities ratio (%)']
latest['No. of recovered'] = latest['Cumulative total cases'] - latest['No. of deaths']

# first few rows
latest.head()

In [ ]:
# load countries map
world_map = gpd.read_file('../input/human-development-index-hdi/countries.geojson')
# avoid Antartica
world_map = world_map[world_map['name']!='Antarctica']
# select only important columns
world_map = world_map[['name', 'continent', 'geometry']]
# first few rows
world_map.head()

## SARS (Severe Acute Respiratory Syndrome)

### Cause
> * SARS coronavirus (SARS-CoV) – virus identified in **2003**. 
> * SARS-CoV is thought to be an animal virus from an as-yet-uncertain animal reservoir, perhaps **bats**, that spread to other animals (**civet cats**)
> * It was first infected humans in the **Guangdong province of southern China** in **2002**.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 14))
sns.set_style('whitegrid')

world_map.plot(ax=ax, color='white', edgecolor='black', alpha=0.4)
ax.set_title('First infected humans', 
             loc='left', fontdict={'fontsize': 24, 
                                   'fontfamily': 'monospace', 
                                   'fontweight': 'bold',
                                   'color': 'black'})

ax.scatter(113.7633, 23.3790, color='orangered', s=200, alpha=0.8)
ax.text(120.7633, 23.3790, 'Guandong, China \nin 2002', 
        fontfamily='monospace', fontsize=12, fontweight='bold',
        color='white', backgroundcolor='black')

ax.set_axis_off()

### Transmission
> * Transmission of SARS-CoV is primarily from person to person. 
> * It appears to have occurred mainly during the second week of illness, which corresponds to the peak of virus excretion in respiratory secretions and stool, and when cases with severe disease start to deteriorate clinically. 
> * Most cases of human-to-human transmission occurred in the health care setting, in the absence of adequate infection control precautions. 
> * Implementation of appropriate infection control practices brought the global outbreak to an end.

### Symptoms
> * Influenza-like and include fever
> * Malaise
> * Myalgia
> * Headache
> * Diarrhoea, and
> * Shivering (rigors). 

### Nature of the disease
> * No individual symptom or cluster of symptoms has proved to be specific for a diagnosis of SARS. 
> * Although fever is the most frequently reported symptom, it is sometimes absent on initial measurement, especially in elderly and immunosuppressed patients.
> * Cough (initially dry), shortness of breath, and diarrhoea are present in the first and/or second week of illness. 
> * Severe cases often evolve rapidly, progressing to respiratory distress and requiring intensive care.

### Vaccines
> * None. Experimental vaccines are under development.

### Incubation period (time interval from infection with the virus to onset of symptoms)
> * Incubation period of SARS-CoV is from 2 to 7 days. 
> * A person infected with SARS-CoV cannot spread the disease until they develop symptoms.

In [ ]:
plot_cal('Incubation period of Ebola is from 2 to 7 days', 2, 7)

### Geographical distribution
> * The disease appeared in November 2002 in the Guangdong province of southern China. 
> * This area is considered as a potential zone of re-emergence of SARS-CoV.
> * SARS-CoV affected 26 countries and resulted in more than 8000 cases in 2003

In [ ]:
fig = px.choropleth(latest, locations="Country/Region", locationmode='country names',
                    color="Cumulative total cases", hover_name="Country/Region", 
                    color_continuous_scale="Sunset", 
                    title='Choropleth map cumulative no. of cases')
fig.update(layout_coloraxis_showscale=True)
fig.show()

# EDA

## Country wise summary

In [ ]:
plot_barh('Cumulative total cases')

In [ ]:
plot_barh('No. of deaths')

In [ ]:
plot_barh('Case fatalities ratio (%)')

### Imported Cases

In [ ]:
plot_barh('Number of Imported cases')

### Health Care Workers

In [ ]:
plot_barh('Number of HCW affected')

In [ ]:
plot_pie('No. of deaths', 'No. of recovered', 'CFR', ['lightseagreen', 'orangered'])

In [ ]:
plot_pie('Cumulative male cases', 'Cumulative female cases', 'Gender wise', ['royalblue', 'crimson'])

In [ ]:
plot_pie('Number of HCW affected', 'Number of non HCW affected', 'HCW', ['whitesmoke', 'dodgerblue'])

In [ ]:
plot_pie('Number of Imported cases', 'Number of locally transmitted cases', 'Imported cases', ['slateblue', 'gold'])

### Onset Date

In [ ]:
latest = latest.sort_values('Date onset first probable case', ascending=False)

country = latest['Country/Region']
start = latest['Date onset first probable case'].to_numpy()
end = latest['Date onset last probable case'].to_numpy()

sns.set_style('whitegrid')
plt.figure(figsize=(15, 12))

plt.hlines(y=country, xmin=start, xmax=end, color='black', alpha=0.8)
plt.scatter(start, country, color='tomato', alpha=1, s=200, label='Date onset first probable case')
plt.scatter(end, country, color='black', alpha=1 , s=200, label='Date onset last probable case')

sns.despine(left=False, bottom=True)
plt.title('Onset Date', loc='left', fontsize=24)
plt.legend(ncol=2)
plt.show()

### Age Range

In [ ]:
age = latest[['Country/Region', 'Median age', 'Age range']]
age = age.sort_values('Median age', ascending=False)
age = age.dropna(subset=['Median age'])        

age['Min age'] = age['Age range'].str.extract('(\d+)')
age['Max age'] = age['Age range'].str.extract('\-(\d+)')

for col in ['Median age', 'Min age', 'Max age']:
    age[col] = pd.to_numeric(age[col])

country = age['Country/Region']
median_age = pd.to_numeric(age['Median age'])
min_age = pd.to_numeric(age['Min age'])
max_age = pd.to_numeric(age['Max age'])

sns.set_style('whitegrid')
plt.figure(figsize=(15, 12))

plt.hlines(y=country, xmin=min_age, xmax=max_age, color='black', alpha=0.8)
plt.scatter(min_age, country, color='dimgray', alpha=1, s=200, label='Minimum Age')
plt.scatter(median_age, country, color='turquoise', alpha=1, s=200, label='Median age')
plt.scatter(max_age, country, color='black', alpha=1 , s=200, label='Maximum Age')

sns.despine(left=False, bottom=True)
plt.title('Age Range', loc='left', fontsize=24)
plt.legend(ncol=3)
plt.show()

# Day wise

In [ ]:
def plot_daywise(col, hue):
    temp = day_wise[day_wise['Date'] > '2003-04-12']
    fig = px.area(temp, x="Date", y=col, width=700, 
                  color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
def plot_stacked(col):
    temp = df[df['Date'] > '2003-04-12']
    fig = px.area(temp, x="Date", y=col, color='Country', 
                 height=600, title=col, 
                 color_discrete_sequence = px.colors.cyclical.mygbm)
    fig.update_layout(showlegend=True)
    fig.show()

In [ ]:
plot_daywise('Cases', 'black')

In [ ]:
plot_daywise('Deaths', 'orangered')

In [ ]:
plot_daywise('Recovered', 'limegreen')

In [ ]:
plot_daywise('Active', 'crimson')

In [ ]:
plot_stacked('Cases')

In [ ]:
plot_stacked('Deaths')

In [ ]:
plot_stacked('Recovered')

In [ ]:
plot_stacked('Active')

In [ ]:
# temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
# temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
#                  var_name='Case', value_name='Count')
# temp.head()

# # fig = px.area(temp, x="Date", y="Count", color='Case', 
#               height=600, width=700, title='Cases over time', 
#               color_discrete_sequence = ['limegreen', 'crimson', 'black'])
# fig.update_layout(xaxis_rangeslider_visible=True)
# fig.show()

In [ ]:
temp = latest[['Country/Region', 'Cumulative male cases', 'Cumulative female cases']]
temp['% Male'] = temp['Cumulative male cases']*100/(temp['Cumulative male cases']+temp['Cumulative female cases'])
temp['% Female'] = temp['Cumulative female cases']*100/(temp['Cumulative male cases']+temp['Cumulative female cases'])
temp.head()

In [ ]:
temp1 = temp.melt(id_vars='Country/Region', 
                 value_vars=['Cumulative male cases', 'Cumulative female cases'], 
                 var_name='Case', value_name='Count')
temp1 = temp1.sort_values(['Count'], ascending=True)


fig = px.bar(temp1, x='Count', y='Country/Region', color='Case', 
             opacity=1, orientation='h', height=600,
             barmode='stack',
             color_discrete_sequence=['indigo', 'deeppink'])
fig.update_layout(title='No. of case based on Gender', xaxis_title="", yaxis_title="")
fig.show()

In [ ]:
temp2 = temp.melt(id_vars='Country/Region', 
                 value_vars=['% Male', '% Female'], 
                 var_name='Case', value_name='Percentage')
temp2 = temp2.sort_values(['Percentage'], ascending=True)

fig = px.bar(temp2, x='Percentage', y='Country/Region', color='Case', 
             opacity=1, orientation='h', height=600,
             barmode='stack',
             color_discrete_sequence=['indigo', 'deeppink'])
fig.update_layout(title='No. of case based on Gender', xaxis_title="", yaxis_title="")
fig.show()

In [ ]:
def plot_overlay_bar(col, hue):
    temp = latest[['Country/Region', 'Cumulative total cases', col]]
    temp = temp.melt(id_vars='Country/Region', 
                     value_vars=['Cumulative total cases', col], 
                     var_name='Case', value_name='Count')
    temp = temp.sort_values(['Case', 'Count'], ascending=True)
    
    fig = px.bar(temp, x='Count', y='Country/Region', color='Case', 
                 opacity=1, orientation='h', height=600,
                 barmode='overlay',
                 color_discrete_sequence=['black', hue])
    fig.update_layout(title=col, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
def plot_overlay_percent_bar(col1, col2, hue):
    temp = latest[['Country/Region', col1, col2]]
    temp = temp.melt(id_vars='Country/Region', 
                     value_vars=[col1, col2], 
                     var_name='Case', value_name='Percentage')
    temp = temp.sort_values(['Case', 'Percentage'], ascending=True)
    
    fig = px.bar(temp, x='Percentage', y='Country/Region', color='Case', 
                 opacity=1, orientation='h', height=600,
                 barmode='stack', range_x=[0,100],
                 color_discrete_sequence=[hue, 'black'])
    fig.update_layout(title=col1, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
plot_overlay_bar('No. of deaths', 'orangered')

In [ ]:
plot_overlay_percent_bar('Case fatalities ratio (%)', 'Non fatality case (%)', 'orangered')

In [ ]:
plot_overlay_bar('Number of Imported cases', 'gold')

In [ ]:
plot_overlay_percent_bar('Percentage of Imported cases', 'Percentage of locally transmitted cases', 'gold')

In [ ]:
plot_overlay_bar('Number of HCW affected', 'cornflowerblue')

In [ ]:
plot_overlay_percent_bar('Percentage of HCW affected', 'Percentage of non HCW affected', 'cornflowerblue')